# Recommender ChatBot: Evaluation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

BASE_PATH    = '../..'
API_PATH     = f'{BASE_PATH}/chat-bot-api'
LIB_PATH     = f'{BASE_PATH}/lib'
DATASET_PATH = f'{BASE_PATH}/datasets'

os.environ['TMP_PATH']         = f'{BASE_PATH}/tmp'
os.environ['DATASET_PATH']     = f'{BASE_PATH}/datasets'
os.environ['WEIGHTS_PATH']     = f'{BASE_PATH}/weights'
os.environ['METRICS_PATH']     = f'{BASE_PATH}/metrics'

In [3]:
import sys
sys.path.append(LIB_PATH)
sys.path.append(API_PATH)

import pytorch_common.util as pu

import client

import util as ut
from faker import Faker

# Common Functions and Classes

In [4]:
import pandas as pd
import numpy as np

def genres_count(interactions_test_set, items, user_id):
    df = interactions_test_set[interactions_test_set['user_id'] == user_id]

    item_ids = df['item_id'].unique()
    
    genres = []
    for item_id in item_ids:
        genres.extend(items[items['movie_id'] == item_id]['movie_genres'].tolist()[0])

    return pd.Series(genres).value_counts().reset_index(name='count')



def releases(interactions_test_set, items, user_id):
    df = interactions_test_set[interactions_test_set['user_id'] == user_id]

    return np.sort(items[items['movie_id'].isin(df['item_id'].unique())]['movie_release_year'].unique())

## Setup

In [5]:
pu.LoggerBuilder().on_console().build()

<RootLogger root (INFO)>

**Step 1**: Load interactions test set.

In [6]:
interactions_test_set = ut.load_df(f'{DATASET_PATH}/interactions_test_set.json')

In [7]:
items = ut.load_df(f'{DATASET_PATH}/pre-processed-movies.json')

**Step 2**: Create **Recommendation Chatbot API** client.

In [8]:
api_client = client.RecChatBotV1ApiClient()

In [14]:
api_client.health

2024-03-18 21:23:16,852 - INFO - GET http://nonosoft.ddns.net:8080/api/v1/health



{
    'chatbot_api': True,
    'ollama_api': True,
    'airflow': {'metadatabase': True, 'scheduler': True},
    'mongo_database': True,
    'choma_database': True
}

**Step 3**: Create profiles for test users.

In [10]:
fake = Faker()

to_email = lambda name: name.lower().replace(' ', '.') + '@gmail.com'

user_ids = interactions_test_set['user_id'].unique()

profiles = []

for user_id in user_ids:
    name  = fake.name()
    email  = to_email(name)
    genres_count_df = genres_count(interactions_test_set, items, user_id=user_id)
    genres = genres_count_df[genres_count_df['count'] >= 10]['index'].tolist()

    release = str(releases(interactions_test_set, items, user_id=user_id)[0])
    
    profile = client.UserProfileDto(
        name             = name,
        email            = to_email(name),
        preferred_from   = release,
        preferred_genres = genres
    )

    profiles.append(profile)

In [12]:
for profile in profiles:
    api_client.remove_interactions_by_user_id(profile.email)

2024-03-18 20:34:57,309 - INFO - DELETE http://nonosoft.ddns.net:8080/api/v1/interactions/users/david.tran@gmail.com
2024-03-18 20:34:57,368 - INFO - DELETE http://nonosoft.ddns.net:8080/api/v1/interactions/users/charles.williams@gmail.com
2024-03-18 20:34:57,395 - INFO - DELETE http://nonosoft.ddns.net:8080/api/v1/interactions/users/marissa.spencer@gmail.com
2024-03-18 20:34:57,417 - INFO - DELETE http://nonosoft.ddns.net:8080/api/v1/interactions/users/pamela.knox@gmail.com
2024-03-18 20:34:57,440 - INFO - DELETE http://nonosoft.ddns.net:8080/api/v1/interactions/users/sabrina.diaz@gmail.com
2024-03-18 20:34:57,463 - INFO - DELETE http://nonosoft.ddns.net:8080/api/v1/interactions/users/george.knight@gmail.com
2024-03-18 20:34:57,485 - INFO - DELETE http://nonosoft.ddns.net:8080/api/v1/interactions/users/latoya.snow@gmail.com
2024-03-18 20:34:57,508 - INFO - DELETE http://nonosoft.ddns.net:8080/api/v1/interactions/users/veronica.taylor@gmail.com
2024-03-18 20:34:57,530 - INFO - DELETE h

In [13]:
for profile in profiles:
    api_client.add_profile(profile)

2024-03-18 20:35:02,672 - INFO - POST http://nonosoft.ddns.net:8080/api/v1/profiles
2024-03-18 20:35:02,700 - INFO - POST http://nonosoft.ddns.net:8080/api/v1/profiles
2024-03-18 20:35:02,726 - INFO - POST http://nonosoft.ddns.net:8080/api/v1/profiles
2024-03-18 20:35:02,749 - INFO - POST http://nonosoft.ddns.net:8080/api/v1/profiles
2024-03-18 20:35:02,771 - INFO - POST http://nonosoft.ddns.net:8080/api/v1/profiles
2024-03-18 20:35:02,795 - INFO - POST http://nonosoft.ddns.net:8080/api/v1/profiles
2024-03-18 20:35:02,817 - INFO - POST http://nonosoft.ddns.net:8080/api/v1/profiles
2024-03-18 20:35:02,841 - INFO - POST http://nonosoft.ddns.net:8080/api/v1/profiles
2024-03-18 20:35:02,864 - INFO - POST http://nonosoft.ddns.net:8080/api/v1/profiles
2024-03-18 20:35:02,884 - INFO - POST http://nonosoft.ddns.net:8080/api/v1/profiles
2024-03-18 20:35:02,907 - INFO - POST http://nonosoft.ddns.net:8080/api/v1/profiles
2024-03-18 20:35:02,929 - INFO - POST http://nonosoft.ddns.net:8080/api/v1/p

In [14]:
%%time

query = {
    "message": {
        "author": profiles[0].email,
        "content": "I want see marvel movies"
    },
    "settings": {
        "llm"                                   : "llama2-7b-chat",
        "retry"                                 : 2,
        "plain"                                 : False,
        "include_metadata"                      : True,
        "rag": {
            "shuffle"                           : True,
            "candidates_limit"                  : 50,
            "llm_response_limit"                : 50,
            "recommendations_limit"             : 5,
            "similar_items_augmentation_limit"  : 5,
            "not_seen": True
        },
        "collaborative_filtering": {
            "shuffle"                           : False,
            "candidates_limit"                  : 50,
            "llm_response_limit"                : 50,
            "recommendations_limit"             : 5,
            "similar_items_augmentation_limit"  : 5,
            "text_query_limit"                  : 5000,
            "k_sim_users"                       : 10,
            "random_selection_items_by_user"    : 0.5,
            "max_items_by_user"                 : 10,
            "min_rating_by_user"                : 3.5,
            "not_seen"                          : True
        }
    }
}

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.96 µs


In [15]:
result = api_client.recommendations(query)
result.items

2024-03-18 20:35:13,710 - INFO - POST http://nonosoft.ddns.net:8080/api/v1/recommendations



[
    
(
    Id         : 102125
    Title      : Iron Man 3
    Release    : 2013
    Genres     : ['action', 'sci-fi', 'thriller', 'imax']
    Rating     : 3.2736842105
    Poster     : http://image.tmdb.org/t/p/w500/qhPtAc1TKbMPqNvcdXSOn9Bn7hZ.jpg
    Description: Action-packed superhero movie with great performances from robert downey jr. and gwyneth paltrow.
)
,
    
(
    Id         : 59315
    Title      : Iron Man
    Release    : 2013
    Genres     : ['action', 'adventure', 'sci-fi']
    Rating     : 3.8469387755
    Poster     : http://image.tmdb.org/t/p/w500/78lPtwv72eTNqFW9COBYI0dWDJa.jpg
    Description: Action-packed superhero movie with great performances from robert downey jr. and gwyneth paltrow.
)
,
    
(
    Id         : 77561
    Title      : Iron Man 2
    Release    : 2013
    Genres     : ['action', 'adventure', 'sci-fi', 'thriller', 'imax']
    Rating     : 3.4307692308
    Poster     : http://image.tmdb.org/t/p/w500/6WBeq4fCfn7AN0o21W9qNcRF2l9.jpg
    Descri

In [16]:
result.items[0].vote(5)
api_client.interactions_by_user(user_id)

2024-03-18 20:35:29,309 - INFO - GET http://nonosoft.ddns.net:8080/api/v1/interactions/make/david.tran@gmail.com/102125/5
2024-03-18 20:35:29,338 - INFO - 204


[]

In [17]:
result = api_client.recommendations(query)
result.items

2024-03-18 20:35:29,377 - INFO - POST http://nonosoft.ddns.net:8080/api/v1/recommendations



[
    
(
    Id         : 2153
    Title      : Avengers, The
    Release    : 2012
    Genres     : ['action', 'adventure']
    Rating     : 1.7777777778
    Poster     : http://image.tmdb.org/t/p/w500/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg
    Description: An action-packed superhero movie featuring iron man, captain america, thor, and other marvel characters.
)
,
    
(
    Id         : 112852
    Title      : Guardians of the Galaxy
    Release    : 2014
    Genres     : ['action', 'adventure', 'sci-fi']
    Rating     : 3.7108433735
    Poster     : http://image.tmdb.org/t/p/w500/r2J02Z2OpNTctfOSN1Ydgii51I3.jpg
    Description: A fun and quirky space adventure with a diverse group of heroes, including star-lord, gamora, drax, rocket, and groot.
)
,
    
(
    Id         : 122918
    Title      : Guardians of the Galaxy 2
    Release    : 2014
    Genres     : ['action', 'adventure', 'sci-fi']
    Rating     : 3.4367816092
    Poster     : http://image.tmdb.org/t/p/w500/y4MBh0EjBlMuOzv9axM